# Trabalho Prático 0 de Estruturas Criptográficas

### Grupo 5
- Duarte Oliveira (pg47157)
- Melânia Pereira (pg47520)

In [89]:
# Imports necessários à execução do código presente neste notebook

import os
import time
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import padding

# Exercício 1

#### Neste exercício é pedido que se crie uma comunicação privada assíncrona entre um agente Emitter e um agente Receiver que cubra alguns aspetos enunciados a seguir.

Começamos por desenvolver uma função para derivar uma chave, baseada no funcionamento do protocolo DH para geração de chaves privadas e públicas. Para este protocolo, é necessário gerar parâmetros que devem ser os mesmos para os dois agentes e podem ser reutilizados.<br>
Inicialmente, gera-se a chave privada e depois é feita a troca com uma chave pública recebida (que será do outro agente com quem se está a comunicar) que irá gerar uma chave partilhada (a mesma nos dois agentes). Esta chave é ainda passada a uma função de derivação de chaves, para permitir uma mistura de informação que a torna mais segura.<br>
Para a chave poder ser gerada, é preciso saber a chave pública do outro agente, para isso, foi também criada uma função que devolve a chave pública gerada.

In [90]:
parameters = dh.generate_parameters(generator=2, key_size=1024)

def get_public_key(private_key):
        return private_key.public_key()

def derivate_key(private_key,public):
        shared_key = private_key.exchange(public)
        
        derived_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=b'handshake data',
        ).derive(shared_key)

        return derived_key

Depois da chave gerada, é o momento de enviar a mensagem para o agente Receiver.<br>
Para isso, é primeiro gerada uma assinatura, usando o HMAC que será concatenada com a mensagem cifrada e enviada ao Receiver para que este possa verificar o remetente.

In [91]:
def auth(message,derived_key):
        h = hmac.HMAC(derived_key, hashes.SHA256())
        h.update(message)
        return h.finalize()

De seguida, é usada a função *urandom()* do pacote *os* do *python* para gerar o *nonce* necessário para cifrar a mensagem.<br>
A cifra usada foi a AESGCM. Depois de tudo gerado e da mensagem cifrada, esta é concatenada à assinatura e ao *nonce* e enviada para o Receiver.

In [92]:
def cifraGCM(nonce,message,derived_key):
    aesgcm = AESGCM(derived_key)
    return aesgcm.encrypt(nonce, message, b'some associated data')

def send_message(message,derived_key):
        signature = auth(b'this is a message to check the signature',derived_key)
        message = message.encode('utf-8')
        nonce = os.urandom(16)
        ct = cifraGCM(nonce,message,derived_key)
        
        return signature + nonce + ct

Do lado do Receiver, os dados são recebidos e desempacotados.

In [93]:
def unpack_data(dados):
      signature = dados[0:32]
      nonce = dados[32:32+16]
      ct = dados[32+16:]

      return signature, nonce, ct

Depois de ter os dados desempacotaos e separados, é necessário fazer a verificação da assinatira da mensagem recebida. Novamente, usando o HMAC.

In [94]:
def verify(signature,derived_key):
      h = hmac.HMAC(derived_key, hashes.SHA256())
      h.update(b'this is a message to check the signature')
      return h.verify(signature)

Se a assinatura for verificada, passa-se então à decifragem da mensagem pela cifra AESGCM.

In [95]:
def decifraGCM(nonce, ct, derived_key):
    aesgcm = AESGCM(derived_key)
    return aesgcm.decrypt(nonce, ct, b'some associated data')

def read_message(ct, derived_key):
      signature, nonce, ct = unpack_data(ct)
      try :
          verify(signature, derived_key)
      except:
          raise Exception("Falha na autenticidade da chave") 

      texto_limpo = decifraGCM(nonce, ct, derived_key)
    
      return texto_limpo.decode('utf-8')

Finalmente, enviamos a mensagem do Emitter para o Receiver.

In [96]:
def main():
    rv_private_key = parameters.generate_private_key()
    em_private_key = parameters.generate_private_key()
    em_derived_key = derivate_key(em_private_key,get_public_key(rv_private_key))
    rv_derived_key = derivate_key(rv_private_key,get_public_key(em_private_key))
    
    dados = send_message("Estruturas Criptográficas é uma unidade curricular do perfil de CSI", em_derived_key)
    print('encrypted text:',dados)

    try:
        pt = read_message(dados,rv_derived_key)

        print('decrypted text:', pt)
    except:
        print("Falha na autenticação da chave")  

main()

encrypted text: b"\xdb\xf4\x7f\xa1\x87\xd9\xb8\x81e\xff:\x97\xc0\xcb@\x81\xa5[\xff:\x0e?\xa5co\xa5\xf8'\xf7\xbc\xa1%\x93B\xc0b\x9a\xe6\x05\xb4*R>\xd1\x1b\xb8\x9f\x9e\x1f,\x86o\xab\x7f\x08\xd4~*T\xd1\xf9\xa2bm\xcap\xb5\x0f\xaa(\xd0J\xb4Z\xe5\xafR\x96\xf3\x9c\xb2p_\xf2s\x938\xbc\xfdZ\xd7\x96\x8a\xb3g}\xe0|\xe2\xa6\xd7\x15\x9bq\xee\xde\x86\xa4\x80\xff\xe8\x04\xc7I\x19\xdaqv\x12-\xd1/4\xba\xcb\x97\xa9\x06o\x0b\xca=\xab"
decrypted text: Estruturas Criptográficas é uma unidade curricular do perfil de CSI


# Exercicio 2
## Criar uma cifra com autenticação de meta-dados a partir de um PRG


Como objetivo inicial, esperava-se que o nosso grupo implementasse um gerador pseudo-aleatório do usando uma função XOF("extended output function") - SHAKE256 - de forma a gerar uma sequência de palavras, cada uma com 64 bits.


Para começar, criamos uma função que recebe a password e a partir de uma *KDF* - no nosso caso decidimos escolher a **PBKDF2HMAC**, mas também poderíamos ter usado a *HKD*. Esta decisão foi tida em conta com a leitura da documentação da lib *Crypthography*, a qual inferia que há diferentes tipos de funções de derivação para diferentes tipos de propósitos. Como o nosso objetivo seria criar uma seed para o gerador, escolhemos um dos que era recomendado. 

In [97]:
def derive_key(password):
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        )

    return kdf.derive(password)

De seguida, e conforme enunciado, era necessário criar o gerador, que tivesse um limite de palavras definido por um valor inicialmente parametrizado como **N**. Com a seed gerada pela *key derivation function* que escolhemos seria possível gerar 2^N palavras de 64 bits cada.
Seguindo o requisitado para este projeto, era necessário usar o **SHAKE256** para definir o gerador - como se sabe o *SHAKE256* é uma *extendable-output function* e define-se o seu uso tendo em conta a necessidade de geração de palavras usando uma XOF. 


In [98]:
def prg(seed,N):
    digest = hashes.Hash(hashes.SHAKE256(8 * pow(2,N))) # sequencia palavras 64 bits / 8 = 8 bytes
    digest.update(seed)
    msg = digest.finalize()
    return msg

A partir da sequência de bits representativa da lista de palavras, gerada pelo PRG, conseguimos aplicar a cifra **OTP**.
Assumindo a variante construída em 1919, é necessário definir a operação *XOR* em contraste com a sua versão original, criada com adição modular. É então preciso aplicar esta operação ao processo de cifragem e decifragem, sendo que, a partir de duas sequências de bytes é aplicado o XOR a ambas.


In [99]:
BLOCK = 8 # bytes; representativo de cada bloco que define uma palavra (64 bits)

In [100]:
def pad_divide(message):
    x = []
    for i in range (0,len(message), BLOCK):
        next = i+BLOCK
        x.append(message[i:next])
    return x

É importante denotar o mecanismo associado a estes processos de cifragem e decifragem. Inicialmente é feito o *padding*, de forma a que os blocos de mensagens sejam garantidamente múltiplos, adicionando informação irrelevante. Após isto, e para simplicidade prática,é necessário dividir a mensagem em blocos de 8 bytes - 64 bits - cada um representando uma palavra.

In [101]:
def cipher(k,msg):
    ciphertext = b''
    pad = padding.PKCS7(64).padder()
    
    padded = pad.update(msg) + pad.finalize()
    p = pad_divide(padded)

    for x in range (len(p)): 
        for bloco, byte in enumerate(p[x]): 
            ciphertext += bytes([byte ^ k[x:(x+1)*BLOCK][bloco]]) 
    return ciphertext


In [102]:

def decipher(k,ciphertext):
    plaintext=b''
    
    p=pad_divide(ciphertext)

    for x in range (len(p)): 
        for bloco, byte in enumerate(p[x]): 
            plaintext += bytes([byte ^ k[x:(x+1)*BLOCK][bloco]]) 
    
   
    unpadder = padding.PKCS7(64).unpadder()
    
    unpadded = unpadder.update(plaintext) + unpadder.finalize()
    return unpadded.decode("utf-8")

De forma a se proceder à autenticação definimos as seguintes funções:

In [103]:
def authentication(ck, message):
        h = hmac.HMAC(ck, hashes.SHA256())
        h.update(message)
        return h.finalize()

def verify_Auth(ck,message, signature):
        h = hmac.HMAC(ck, hashes.SHA256())
        h.update(message)
        try: 
            h.verify(signature)
            return True
        except cryptography.exceptions.InvalidSignature:
            return False

### Verifica-se então de seguida todo o funcionamento prático deste exercício, e respetivos resultados:

In [105]:
N = 4 # VALOR DO PARAMETRO N

cipher_key = derive_key(b'password')
gerado = prg(cipher_key,N)
mensagem = b"Super hyper mega ultra secret message"

ct = cipher(gerado,mensagem)
auth = authentication(cipher_key,ct)
if verify_Auth(cipher_key,ct, auth):
    dt = decipher(gerado,ct)
else:
    print("Falha na autenticação do criptograma!")


print("Plain text: ", mensagem)
print("Cipher text:  ", ct)
print("Clear text:  ", dt)

Plain text:  b'Super hyper mega ultra secret message'
Cipher text:   b'%\xe2:f\xda\x9b9\x8d\xe7/q\x88\xd64\x93\x8ajv\xc4\xcf#\x95\xcbNf\xcb\xc94\x80\xcbP{\xdb\xc80\x93\x8e>\x1d\xcb'
Clear text:   Super hyper mega ultra secret message


# Exercício 3

Para proceder à comparação das duas cifras, desenvolveu-se um cenário de teste onde é testado o tempo que cada uma das cifras demora a cifrar o mesmo texto.

In [107]:
N = 15
    
def fst_cipher():
    pwd = b"password"
    key = derive_key(pwd)
    nonce = os.urandom(16)
    plaintext = os.urandom(2 ** N)
    ciphertext = cifraGCM(nonce, plaintext, key)
    return ciphertext

def snd_cipher():
    pwd = b"password"
    key = derive_key(pwd)
    plaintext = os.urandom(2 ** N)
    words = prg(key,N)
    ciphertext = cipher(words,plaintext)
    return ciphertext

fst = []
snd = []
for i in range(1,100):
    start = time.time_ns()
    fst_text = fst_cipher()
    stop = time.time_ns()
    fst.append(stop-start)

    start = time.time_ns()
    snd_text = snd_cipher()
    stop = time.time_ns()
    snd.append(stop-start)

print('avg elapsed time fst:', sum(fst)/len(fst), 'ns')
print('avg elapsed time snd:', sum(snd)/len(snd), 'ns')

avg elapsed time fst: 48056515.15151515 ns
avg elapsed time snd: 91401050.50505051 ns


Correndo a célula anterior podemos, então, reparar que a cifra desenvolvida por nós (no problema 2) é mais ineficiente do que a cifra AESCGM (usada no problema 1).<br>
No entanto, podemos ainda, alterando o valor de N, constatar que isto é algo que não acontece sempre, pois, para valores de N mais pequenos, a cifra desenvolvida por nós torna-se mais eficiente. Aliás, se aumentarmos o valor de N, podemos ainda perceber que esta última cifra perde eficiência a um nível exponencial. <br>
Apesar da cifra AESCGM ser mais eficiente, podem existir casos em que a eficiência nao seja o mais importante, sendo que, nesses, a cifra desenvolvida no problema 2 é mais adequada. <br>
No segundo algoritmo, podemos considerar que a geração do *par* pode ser um *bottleneck*, enquanto que no primeiro se pode considerar como *bottleneck* o facto de, no Emitter, ser necessário gerar uma chave por cada mensagem, assim como o seu código de autenticação e ainda cifrar o criptograma e, no Receiver, ainda o trabalho adicional de autenticar a chave que foi recebida na mensagem.